In [1]:
import ee,geemap

# ee.Initialize()

Map = geemap.Map()

point = ee.Geometry.Point([-122.4439, 37.7538])
# 建立一个点坐标
# point = ee.Geometry.Point([-87.7719, 41.8799])

# 导入landsat8数据
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
# Map

In [2]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
# Map

In [3]:
# 提前特征值数据
# 选取样点
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

Map.addLayer(points, {}, 'training', False)
# Map

In [4]:
# 提取目标值数据
label = 'landcover'

training = image.sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})
# 使用Cart(分类与回归树)
trained = ee.Classifier.smileCart().train(training, label)

In [5]:
# 将分类器应用于地图上
result = image.classify(trained)

Map.addLayer(result.randomVisualizer(), {}, 'classfied')
# Map

In [8]:
# 通过值来改变颜色

# 获取值
class_values = nlcd.get('landcover_class_values').getInfo()
# 获取值对应的颜色
class_palette = nlcd.get('landcover_class_palette').getInfo()
# 改变图像对应的属性
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
nlcd.propertyNames().getInfo()

['system:footprint',
 'impervious_descriptor_class_values',
 'landcover_class_names',
 'system:time_end',
 'system:id',
 'landcover_class_values',
 'system:time_start',
 'impervious_descriptor_class_palette',
 'impervious_descriptor_class_names',
 'system:version',
 'system:asset_size',
 'landcover_class_palette',
 'system:index',
 'system:bands',
 'system:band_names']

In [13]:
nlcd.get('landcover_class_palette').getInfo()

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']